# Capstone project: Final assignment week 5

# Code

### Visualization of the locations with Folium

In [22]:
import pandas as pd
import numpy as np
import requests # library to handle requests
from bs4 import BeautifulSoup
import copy
import json # library to handle JSON files

#!conda install -c conda-forge geopy --yes # uncomment this line if you haven't completed the Foursquare API lab
from geopy.geocoders import Nominatim # convert an address into latitude and longitude values

import requests # library to handle requests
from pandas.io.json import json_normalize # tranform JSON file into a pandas dataframe

# Matplotlib and associated plotting modules
import matplotlib.cm as cm
import matplotlib.colors as colors

# import k-means from clustering stage
from sklearn.cluster import KMeans

#!conda install -c conda-forge folium=0.5.0 --yes # uncomment this line if you haven't completed the Foursquare API lab
import folium # map rendering library

print('Libraries imported.')

Libraries imported.


### Some useful functions for getting the data and the visualization

In [33]:
# function that extracts the category of the venue
def get_category_type(row):
    try:
        categories_list = row['categories']
    except:
        categories_list = row['venue.categories']
        
    if len(categories_list) == 0:
        return None
    else:
        return categories_list[0]['name']

In [34]:
def get_congitute_latitude_accomodation(address):
    geolocator = Nominatim(user_agent="foursquare_agent")
    location = geolocator.geocode(address)
    latitude = location.latitude
    longitude = location.longitude
    return latitude, longitude

In [35]:
def filter_data(dataframe):
    # keep only columns that include venue name, and anything that is associated with location
    filtered_columns = ['name', 'categories'] + [col for col in dataframe.columns if col.startswith('location.')] + ['id']
    dataframe_filtered = dataframe.loc[:, filtered_columns]

    # filter the category for each row
    dataframe_filtered['categories'] = dataframe_filtered.apply(get_category_type, axis=1)

    # clean column names by keeping only last term
    dataframe_filtered.columns = [column.split('.')[-1] for column in dataframe_filtered.columns]

    return dataframe_filtered

In [36]:
def query_into_dataframe(CLIENT_ID, CLIENT_SECRET, address,ACCESS_TOKEN, VERSION, search_query, radius, LIMIT):
    latitude, longitude = get_congitute_latitude_accomodation(address)
    
    url = 'https://api.foursquare.com/v2/venues/search?client_id={}&client_secret={}&ll={},{}&oauth_token={}&v={}&query={}&radius={}&limit={}'.format(CLIENT_ID, CLIENT_SECRET, latitude, longitude,ACCESS_TOKEN, VERSION, search_query, radius, LIMIT)
    results = requests.get(url).json()
    
    # assign relevant part of JSON to venues
    venues = results['response']['venues']

    # tranform venues into a dataframe
    dataframe = pd.json_normalize(venues)   
    return dataframe, latitude, longitude

In [37]:
def plot_map(latitude, longitude, dataframe_filtered):
    venues_map = folium.Map(location=[latitude, longitude], zoom_start=15) # generate map centred around the place rented

    # add a red circle marker to represent the place rented
    folium.CircleMarker(
        [latitude, longitude],
        radius=10,
        color='red',
        popup='Rented place',
        fill = True,
        fill_color = 'red',
        fill_opacity = 0.6
    ).add_to(venues_map)

    # add the Italian restaurants as blue circle markers
    for lat, lng, label in zip(dataframe_filtered.lat, dataframe_filtered.lng, dataframe_filtered.categories):
        folium.CircleMarker(
            [lat, lng],
            radius=5,
            color='blue',
            popup=label,
            fill = True,
            fill_color='blue',
            fill_opacity=0.6
        ).add_to(venues_map)

    # display map
    return venues_map

In [75]:
def plot_map_2colors(latitude, longitude, dataframe_filtered, colors):
    venues_map = folium.Map(location=[latitude, longitude], zoom_start=15) # generate map centred around the place rented

    # add a red circle marker to represent the place rented
    folium.CircleMarker(
        [latitude, longitude],
        radius=10,
        color='red',
        popup='Rented place',
        fill = True,
        fill_color = 'red',
        fill_opacity = 0.6
    ).add_to(venues_map)

    # add the Italian restaurants as blue circle markers
    for lat, lng, label, c in zip(dataframe_filtered.lat, dataframe_filtered.lng, dataframe_filtered.categories, colors):
        folium.CircleMarker(
            [lat, lng],
            radius=5,
            color=c,
            popup=label,
            fill = True,
            fill_color=c,
            fill_opacity=0.6
        ).add_to(venues_map)

    # display map
    return venues_map

#### Visualise Rome address and closeby pizza locations

In [47]:
address = "33 Via di Sant'Agata de Goti, Roma, RM"
search_query = 'Pizza'
radius = 500 #500 meters
df_Rome, latitude_Rome, longitude_Rome = query_into_dataframe(CLIENT_ID, CLIENT_SECRET, address,ACCESS_TOKEN, VERSION, search_query,radius,LIMIT)
df_Rome = filter_data(df_Rome)
venues_map = plot_map(latitude_Rome, longitude_Rome, df_Rome)
venues_map.save('./Rome_pizzas.html')

**html available in the Github directory**

#### Visualise Naples address and closeby pizza locations

In [ ]:
address = '133, Via Stella, Napoli, NA'
search_query = 'Pizza'
radius = 500 #500 meters
df_Naples, latitude_Naples, longitude_Naples = query_into_dataframe(CLIENT_ID, CLIENT_SECRET, address,ACCESS_TOKEN, VERSION, search_query,radius,LIMIT)
df_Naples = filter_data(df_Naples)
venues_map = plot_map(latitude_Naples, longitude_Naples, df_Naples)
venues_map.save('./Naples_pizzas.html')

**html available in the Github directory**

**There are no pizza places in 500 meters, we redo the analysis in 1000 meters**

In [49]:
address = '133, Via Stella, Napoli, NA'
search_query = 'Pizza'
radius = 1000 #500 meters
df_Naples, latitude_Naples, longitude_Naples = query_into_dataframe(CLIENT_ID, CLIENT_SECRET, address,ACCESS_TOKEN, VERSION, search_query,radius,LIMIT)
df_Naples = filter_data(df_Naples)
venues_map = plot_map(latitude_Naples, longitude_Naples, df_Naples)
venues_map.save('./Naples_pizzas.html')

#### Visualise Florence address and closeby pizza locations

In [50]:
address = '3, Via Del Leone, Firenze, FI'
search_query = 'Pizza'
radius = 500 #500 meters
df_Florence, latitude_Florence, longitude_Florence = query_into_dataframe(CLIENT_ID, CLIENT_SECRET, address,ACCESS_TOKEN, VERSION, search_query,radius,LIMIT)
df_Florence = filter_data(df_Florence)
venues_map = plot_map(latitude_Florence, longitude_Florence, df_Florence)
venues_map.save('./Florence_pizzas.html')

**html available in the Github directory**

### Some useful functions for finding the best pizza location

In [14]:
def find_rating_venue(venue_id, CLIENT_ID, CLIENT_SECRET,ACCESS_TOKEN, VERSION):
    url = 'https://api.foursquare.com/v2/venues/{}?client_id={}&client_secret={}&oauth_token={}&v={}'.format(venue_id, CLIENT_ID, CLIENT_SECRET,ACCESS_TOKEN, VERSION)
    result = requests.get(url).json()
    try:
        return result['response']['venue']['rating']
    except:
        return -1#print('This venue has not been rated yet.')

#### Find best rated pizza location in Rome

In [41]:
df_Rome.columns

Index(['name', 'categories', 'address', 'lat', 'lng', 'labeledLatLngs',
       'distance', 'cc', 'city', 'state', 'country', 'formattedAddress',
       'postalCode', 'neighborhood', 'id'],
      dtype='object')

In [42]:
id_list = df_Rome.id.to_list()
ratings_list = []
for i in range(len(names_list)):
    ratings_list.append(find_rating_venue(id_list[i], CLIENT_ID, CLIENT_SECRET,ACCESS_TOKEN, VERSION))

In [43]:
df_Rome_pizzas = copy.deepcopy(df_Rome)
df_Rome_pizzas['ratings'] = ratings_list
df_Rome_pizzas.head()

,name,categories,address,lat,lng,labeledLatLngs,distance,cc,city,state,country,formattedAddress,postalCode,neighborhood,id,ratings
0,Pizza & Mortadella,Pizza Place,Via Cavour,41.893873,12.490045,"[{'label': 'display', 'lat': 41.89387250170169...",156,IT,Roma,Lazio,Italia,"[Via Cavour, Roma Lazio, Italia]",NaN,NaN,4cefd14ef368b1f74bae87f7,-1
1,Pizza €,Pizza Place,NaN,41.895211,12.490705,"[{'label': 'display', 'lat': 41.89521101714234...",112,IT,Roma,Lazio,Italia,"[Roma Lazio, Italia]",NaN,NaN,4e7e02e402d5c3c40122aa7d,8
2,Pizza E Kebab,Taco Place,NaN,41.893618,12.489304,"[{'label': 'display', 'lat': 41.893618, 'lng':...",174,IT,NaN,NaN,Italia,[Italia],NaN,NaN,520c024111d23b4e67d734ed,-1
3,Listino pizza a taglio,Pizza Place,NaN,41.894201,12.491137,"[{'label': 'display', 'lat': 41.89420091702605...",184,IT,NaN,NaN,Italia,[Italia],NaN,NaN,4e7b39fcb61c3d36307fc71a,5
4,La Pizza di Angelino,Pizza Place,NaN,41.893377,12.488152,"[{'label': 'display', 'lat': 41.89337712112877...",223,IT,NaN,NaN,Italia,[Italia],NaN,NaN,4d9afa603a32a09392518719,8


In [44]:
df_Rome_pizzas[['name', 'id', 'distance','ratings']].sort_values(by='distance')[:5]

,name,id,distance,ratings
1,Pizza €,4e7e02e402d5c3c40122aa7d,112,8
0,Pizza & Mortadella,4cefd14ef368b1f74bae87f7,156,-1
15,Pizza's House,5a894f913fcee849ab3b7b93,163,-1
2,Pizza E Kebab,520c024111d23b4e67d734ed,174,-1
3,Listino pizza a taglio,4e7b39fcb61c3d36307fc71a,184,5


Selected pizza location has ID 

In [45]:
ID_selected = '4e7e02e402d5c3c40122aa7d'

In [46]:
df_Rome_pizzas[df_Rome_pizzas.id==ID_selected]

,name,categories,address,lat,lng,labeledLatLngs,distance,cc,city,state,country,formattedAddress,postalCode,neighborhood,id,ratings
1,Pizza €,Pizza Place,NaN,41.895211,12.490705,"[{'label': 'display', 'lat': 41.89521101714234...",112,IT,Roma,Lazio,Italia,"[Roma Lazio, Italia]",NaN,NaN,4e7e02e402d5c3c40122aa7d,8


In [53]:
venues_map = plot_map(latitude_Rome, longitude_Rome, df_Rome_pizzas[df_Rome_pizzas.id==ID_selected])
venues_map.save('./Rome_pizzas_selected.html')

#### Find best rated pizza location in Naples

In [54]:
df_Naples.columns

Index(['name', 'categories', 'lat', 'lng', 'labeledLatLngs', 'distance',
       'postalCode', 'cc', 'city', 'state', 'country', 'formattedAddress',
       'address', 'id'],
      dtype='object')

In [57]:
id_list = df_Naples.id.to_list()
ratings_list = []
for i in range(len(id_list)):
    ratings_list.append(find_rating_venue(id_list[i], CLIENT_ID, CLIENT_SECRET,ACCESS_TOKEN, VERSION))

In [58]:
df_Naples_pizzas = copy.deepcopy(df_Naples)
df_Naples_pizzas['ratings'] = ratings_list
df_Naples_pizzas.head()

,name,categories,lat,lng,labeledLatLngs,distance,postalCode,cc,city,state,country,formattedAddress,address,id,ratings
0,Pizza & Pummarol,Pizza Place,40.850851,14.256835,"[{'label': 'display', 'lat': 40.850851, 'lng':...",908,80138,IT,Napoli,Campania,Italia,"[80138 Napoli Campania, Italia]",NaN,5adb0ad7d807ee19fcc9185f,7
1,Pronto Pizza,Pizza Place,40.849528,14.249854,"[{'label': 'display', 'lat': 40.849528, 'lng':...",752,80135,IT,Napoli,Campania,Italia,"[Via Tarsia, 12, 80135 Napoli Campania, Italia]","Via Tarsia, 12",4da4516d7ccc816e86b58e7b,-1
2,Pizza Loca,Fast Food Restaurant,40.846135,14.249461,"[{'label': 'display', 'lat': 40.84613513529935...",1125,80134,IT,Napoli,Campania,Italia,"[Via Toledo, 72, 80134 Napoli Campania, Italia]","Via Toledo, 72",4fe2282de4b0bb952f791cfd,-1
3,I Maestri Della Pizza,Pizza Place,40.849927,14.249760,"[{'label': 'display', 'lat': 40.84992732478569...",706,NaN,IT,Napoli,Campania,Italia,"[Napoli Campania, Italia]",NaN,4efc5e4cf9ab0847f9ee2603,5
4,Naples Pizza Hostel,Hostel,40.852202,14.256099,"[{'label': 'display', 'lat': 40.85220215545502...",766,NaN,IT,Napoli,80138,Italia,"[44 Via San Paolo ai Tribunali, Napoli 80138, ...",44 Via San Paolo ai Tribunali,4c2871a5360cef3b2b12bedc,-1


In [61]:
df_Naples_pizzas[['name', 'id', 'distance','ratings']].sort_values(by='distance')#[:10]

,name,id,distance,ratings
3,I Maestri Della Pizza,4efc5e4cf9ab0847f9ee2603,706,5
1,Pronto Pizza,4da4516d7ccc816e86b58e7b,752,-1
4,Naples Pizza Hostel,4c2871a5360cef3b2b12bedc,766,-1
9,PizzaBrà,4f098e8be4b06fab1a5cb2c8,831,-1
7,Pizzeria Sorbillo,4b92fe3af964a520382d34e3,859,-1
8,Pizzasleep,58a7e46684c4ed19f839207c,864,-1
5,Antica Pizza Fritta Da Zia Esterina Sorbillo,5ccebb75234724002c911802,879,5
0,Pizza & Pummarol,5adb0ad7d807ee19fcc9185f,908,7
10,Il Pizzaiolo del Presidente,4ccd63bac9b8468814c7c2c3,1015,7
11,Fiorenzano Pizzaioli Dal 1897,5a9d9cdfe075506f4a08cf0a,1028,-1


Selected pizza location has ID 

In [64]:
ID_selected1 = '4da4516d7ccc816e86b58e7b'
ID_selected2 = '5adb0ad7d807ee19fcc9185f'

In [91]:
df_Naples_pizzas[(df_Naples_pizzas.id==ID_selected1)|(df_Naples_pizzas.id==ID_selected2)]

,name,categories,lat,lng,labeledLatLngs,distance,postalCode,cc,city,state,country,formattedAddress,address,id,ratings
0,Pizza & Pummarol,Pizza Place,40.850851,14.256835,"[{'label': 'display', 'lat': 40.850851, 'lng':...",908,80138,IT,Napoli,Campania,Italia,"[80138 Napoli Campania, Italia]",NaN,5adb0ad7d807ee19fcc9185f,7
1,Pronto Pizza,Pizza Place,40.849528,14.249854,"[{'label': 'display', 'lat': 40.849528, 'lng':...",752,80135,IT,Napoli,Campania,Italia,"[Via Tarsia, 12, 80135 Napoli Campania, Italia]","Via Tarsia, 12",4da4516d7ccc816e86b58e7b,-1


In [90]:
venues_map = plot_map_2colors(latitude_Naples, longitude_Naples, df_Naples_pizzas[(df_Naples_pizzas.id==ID_selected1)|
                                                                          (df_Naples_pizzas.id==ID_selected2)], ['green','blue'])
venues_map.save('./Naples_pizzas_selected.html')

#### Find best rated pizza location in Florence

In [79]:
df_Florence.columns

Index(['name', 'categories', 'address', 'crossStreet', 'lat', 'lng',
       'labeledLatLngs', 'distance', 'postalCode', 'cc', 'neighborhood',
       'city', 'state', 'country', 'formattedAddress', 'id'],
      dtype='object')

In [82]:
id_list = df_Florence.id.to_list()
ratings_list = []
for i in range(len(id_list)):
    ratings_list.append(find_rating_venue(id_list[i], CLIENT_ID, CLIENT_SECRET,ACCESS_TOKEN, VERSION))

In [83]:
df_Florence_pizzas = copy.deepcopy(df_Florence)
df_Florence_pizzas['ratings'] = ratings_list
df_Florence_pizzas.head()

,name,categories,address,crossStreet,lat,lng,labeledLatLngs,distance,postalCode,cc,neighborhood,city,state,country,formattedAddress,id,ratings
0,Gusta Pizza,Pizza Place,Via Maggio 46r,Sdrucciolo dei Pitti,43.766584,11.248758,"[{'label': 'display', 'lat': 43.76658430724039...",559,50125,IT,Oltrarno,Firenze,Toscana,Italia,"[Via Maggio 46r (Sdrucciolo dei Pitti), 50125 ...",4b9d3668f964a520f99936e3,-1
1,Il Borgo Della Pizza,Pizza Place,Borgo San Frediano,NaN,43.770023,11.242569,"[{'label': 'display', 'lat': 43.77002309454146...",73,NaN,IT,NaN,Firenze,Toscana,Italia,"[Borgo San Frediano, Firenze Toscana, Italia]",4db302806e8179a9135f3687,-1
2,Pizza Bike,Pizza Place,NaN,NaN,43.774117,11.242652,"[{'label': 'display', 'lat': 43.77411724254026...",519,NaN,IT,NaN,NaN,NaN,Italia,[Italia],5054c601e4b0cbb61a802454,7
3,Runner Pizza,Food Court,NaN,NaN,43.764349,11.244248,"[{'label': 'display', 'lat': 43.764349, 'lng':...",577,NaN,IT,NaN,NaN,NaN,Italia,[Italia],5764642c498eee38910a6572,-1
4,Via Della Fonderia Pizza & Restaurant,Italian Restaurant,Via Della Fonderia 61A,NaN,43.772756,11.237337,"[{'label': 'display', 'lat': 43.772756, 'lng':...",587,50142,IT,NaN,Firenze,Toscana,Italia,"[Via Della Fonderia 61A, 50142 Firenze Toscana...",5bafd279aa6c95002cab8544,-1


In [84]:
df_Florence_pizzas[['name', 'id', 'distance','ratings']].sort_values(by='distance')[:5]

,name,id,distance,ratings
1,Il Borgo Della Pizza,4db302806e8179a9135f3687,73,-1
6,"Trattoria Pizzaria ""IL RISORGIMENTO""",54fcbed8498e23c1e6a8d302,92,-1
2,Pizza Bike,5054c601e4b0cbb61a802454,519,7
0,Gusta Pizza,4b9d3668f964a520f99936e3,559,-1
3,Runner Pizza,5764642c498eee38910a6572,577,-1


Selected pizza location has ID 

In [85]:
ID_selected = '5054c601e4b0cbb61a802454'

In [86]:
df_Florence_pizzas[df_Florence_pizzas.id==ID_selected]

,name,categories,address,crossStreet,lat,lng,labeledLatLngs,distance,postalCode,cc,neighborhood,city,state,country,formattedAddress,id,ratings
2,Pizza Bike,Pizza Place,NaN,NaN,43.774117,11.242652,"[{'label': 'display', 'lat': 43.77411724254026...",519,NaN,IT,NaN,NaN,NaN,Italia,[Italia],5054c601e4b0cbb61a802454,7


In [89]:
venues_map = plot_map(latitude_Florence, longitude_Florence, df_Florence_pizzas[df_Florence_pizzas.id==ID_selected])
venues_map.save('./Florence_pizzas_selected.html')